#### Modelo binario
---

Lo que te dice:
- AUC-ROC: Qué tan bien separa personas neutrales (0) vs con opinión (≠0).
- Coeficientes: Cuánto aumenta/disminuye la probabilidad de tener opinión por cada variable.
- Odds ratios: Por cada variable significativa, cuántas veces más probable es tener opinión
- Variables significativas: Qué factores predicen "tener opinión vs. ser neutral".

Lo que se pierde:
- Dirección de la opinión (Positiva/Negativa).
- Intensidad de la opinión (Leve/Fuerte).
- Diferencias entre tipos de opinión.

In [ ]:
def Modelo_Binario(Variables_X, Variable_Y, Nombre_Variable, 
                            Tipo_Balanceado='normal', Umbral_Decision=0.5):
    
    """
    Modelo binario unificado que permite diferentes configuraciones.
    
    Parámetros:
    - Tipo_Balanceado: 'normal', 'balanced', 'auto'
    - Umbral_Decision: float entre 0 y 1 para clasificación
    
    """
    
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
    
    print(f"\nModelo BINARIO ({Tipo_Balanceado.upper()}) para: {Nombre_Variable}")
    print(f"Umbral de decisión: {Umbral_Decision}")
    print("="*60)
    
    # Crear variable binaria.
    Variable_Binaria = (Variable_Y != 0).astype(int)
    
    print(f"Distribución binaria:")
    print(f"  Neutral (0): {(Variable_Binaria == 0).sum()} obs")
    print(f"  Con opinión (1): {(Variable_Binaria == 1).sum()} obs")
    
    # Configurar class_weight según tipo.
    if Tipo_Balanceado == 'balanced':
        Pesos_Clases = 'balanced'
    elif Tipo_Balanceado == 'auto':
        # Calcular pesos personalizados.
        from sklearn.utils.class_weight import compute_class_weight
        Clases_Unicas = np.unique(Variable_Binaria)
        Pesos_Calculados = compute_class_weight('balanced', 
                                               classes=Clases_Unicas, 
                                               y=Variable_Binaria)
        Pesos_Clases = dict(zip(Clases_Unicas, Pesos_Calculados))
    else:
        Pesos_Clases = None
    
    # Ajustar modelo.
    Modelo_Logistico = LogisticRegression(
        max_iter=1000, 
        random_state=42,
        class_weight=Pesos_Clases
    )
    
    Modelo_Logistico.fit(Variables_X, Variable_Binaria)
    
    # Predicciones con umbral personalizado.
    Probabilidades = Modelo_Logistico.predict_proba(Variables_X)[:, 1]
    Predicciones = (Probabilidades >= Umbral_Decision).astype(int)
    
    # Métricas.
    AUC = roc_auc_score(Variable_Binaria, Probabilidades)
    Precision_General = (Predicciones == Variable_Binaria).mean()
    
    print(f"AUC-ROC: {AUC:.4f}")
    print(f"Precisión general: {Precision_General:.4f}")
    
    print("\nReporte detallado por clase:")
    Reporte = classification_report(Variable_Binaria, Predicciones, 
                                   target_names=['Neutral', 'Con_Opinion'],
                                   output_dict=True)
    print(classification_report(Variable_Binaria, Predicciones, 
                              target_names=['Neutral', 'Con_Opinion']))
    
    print("\nMatriz de confusión:")
    print("Predicho:  Neutral  Con_Opinion")
    Matriz = confusion_matrix(Variable_Binaria, Predicciones)
    print(f"Neutral      {Matriz[0,0]:4d}     {Matriz[0,1]:4d}")
    print(f"Con_Opinion  {Matriz[1,0]:4d}     {Matriz[1,1]:4d}")
    
    # Calcular métricas específicas de clase minoritaria.
    Recall_Minoritaria = Reporte['Con_Opinion']['recall']
    F1_Minoritaria = Reporte['Con_Opinion']['f1-score']
    
    print(f"\nMétricas clase minoritaria (Con_Opinion):")
    print(f"  Recall: {Recall_Minoritaria:.3f}")
    print(f"  F1-Score: {F1_Minoritaria:.3f}")
    
    return {
        'Tipo_Modelo': f'Binario_{Tipo_Balanceado}',
        'Umbral_Decision': Umbral_Decision,
        'AUC': AUC,
        'Precision_General': Precision_General,
        'Recall_Minoritaria': Recall_Minoritaria,
        'F1_Minoritaria': F1_Minoritaria,
        'Matriz_Confusion': Matriz,
        'Modelo': Modelo_Logistico,
        'Reporte_Completo': Reporte
    }